Importações iniciais


In [1]:
import ajuste as aj


Dados subindo:

In [2]:
path = "RC-01.csv"

dado_sub = aj.slicing_data(path,[2,12],ascending=True) #Dados filtrados

dado_sub_regres, b = aj.regres_exp(dado_sub) #Dados da regressão e o parâmetro b

(aj.data_x_regression(dado_sub,dado_sub_regres)).display() #Gráfico

tal_sb = 1/(b*1000)


alt.LayerChart(...)

Dado descendo

In [3]:
dado_dsc = aj.slicing_data(path, [1.05,2.6])

dado_dsc_regres, b = aj.regres_exp(dado_dsc)

(aj.data_x_regression(dado_dsc,dado_dsc_regres)).display()

tal_dc = 1/(b*1000)

alt.LayerChart(...)

In [4]:
with open("RC-01.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb}\n")
    m.write(f"tal_descendo com ajuste: {tal_dc}\n")

In [5]:
path = "RC-02.csv"

data_dsc2 = aj.slicing_data(path,[1,3])

data_dsc2_regres, b = aj.regres_exp(data_dsc2)

(aj.data_x_regression(data_dsc2,data_dsc2_regres)).display()



tal_dsc2 = 1/(b*1000)


alt.LayerChart(...)

In [6]:
path = "RC-02.csv"

data_sb2 = aj.slicing_data(path,[3,10],ascending=True)

data_sb2_regres, b = aj.regres_exp(data_sb2)

(aj.data_x_regression(data_sb2,data_sb2_regres)).display()


tal_sb2 = 1/(b*1000)



alt.LayerChart(...)

In [7]:
with open("RC-02.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb2}\n")
    m.write(f"tal_descendo com ajuste: {tal_dsc2}\n")
    

print("Arquivo escrito com sucesso!")


Arquivo escrito com sucesso!
